In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./dataset/'):
    for filename in filenames:
        if filename.endswith(".csv"):
            print(os.path.join(dirname, filename))

For complying with data sharing policy of the original data owners, some outputs are ommitted.

# Loading Datasets

In [ ]:
train_df=pd.read_csv("dataset/Tamil/train-20250101T183816Z-001/train/train.csv")
train_df.head()

In [ ]:
val_df=pd.read_csv("dataset/Tamil/dev-20250101T183837Z-001/dev/dev.csv")
val_df.head()

In [ ]:
test_df=pd.read_csv("dataset/Tamil/test-20250101T183840Z-001/test/test.csv")
test_df.head()

# Processing

In [32]:
import os
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from transformers import AutoModel, AutoTokenizer, AutoProcessor
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm

In [33]:
# Paths
TRAIN_IMAGE_FOLDER = "dataset/Tamil/train-20250101T183816Z-001/train/"
VAL_IMAGE_FOLDER = "dataset/Tamil/dev-20250101T183837Z-001/dev/"
TEST_IMAGE_FOLDER = "dataset/Tamil/test-20250101T183840Z-001/test/"

In [34]:
# Parameters
IMAGE_MODEL_NAME = "zer0int/CLIP-GmP-ViT-L-14"  # Example vision model
TEXT_MODEL_NAME = "PosteriorAI/dravida_llama2_7b"  # Example Language model
EMBEDDINGS_DIR = "./embeddings"  # Directory to save embeddings
BATCH_SIZE = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [35]:
# Ensure embedding directory exists
os.makedirs(EMBEDDINGS_DIR, exist_ok=True)

In [36]:
# Feature extraction classes for images and text
class ImageDataset(Dataset):
    def __init__(self, image_folder, df, processor):
        self.image_folder = image_folder
        self.df = df
        self.processor = processor
        self.transforms = Compose([
            Resize((224, 224)),
            ToTensor(),
            Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_id = self.df.iloc[idx]["image_id"]
        image_path = os.path.join(self.image_folder, f"{image_id}.jpg")
        image = self.transforms(self.processor(image_path))
        return image, image_id

In [37]:
class TextDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        transcription = self.df.iloc[idx]["transcriptions"]
        transcription = transcription if isinstance(transcription, str) else ""
        inputs = self.tokenizer(
            transcription, padding="max_length", truncation=True, max_length=128, return_tensors="pt"
        )
        return inputs, self.df.iloc[idx]["image_id"]

In [38]:
# Load Hugging Face models
vision_model = AutoModel.from_pretrained(IMAGE_MODEL_NAME).to(device)
vision_processor = AutoProcessor.from_pretrained(IMAGE_MODEL_NAME)
text_model = AutoModel.from_pretrained(TEXT_MODEL_NAME).to(device)
text_tokenizer = AutoTokenizer.from_pretrained(TEXT_MODEL_NAME)

Loading checkpoint shards: 100%|██████████| 3/3 [00:09<00:00,  3.16s/it]


In [39]:
from PIL import Image

def extract_image_embeddings(image_folder, df, save_path, model, processor):
    if os.path.exists(save_path):
        print(f"Embeddings already exist at {save_path}")
        return torch.load(save_path)

    embeddings = {}
    model.eval()
    with torch.no_grad():
        for idx, row in tqdm(df.iterrows(), desc="Extracting image embeddings", total=len(df)):
            image_id = row["image_id"]
            image_path = os.path.join(image_folder, f"{image_id}.jpg")
            
            if not os.path.exists(image_path):
                print(f"Image not found: {image_path}")
                continue

            try:
                # Open image with PIL
                image = Image.open(image_path).convert("RGB")
                # Preprocess image
                inputs = processor(images=image, return_tensors="pt").to(device)
                # Extract embeddings
                outputs = model.get_image_features(**inputs)
                embeddings[image_id] = outputs.cpu()
            except Exception as e:
                print(f"Error processing image {image_id}: {e}")
                continue

    torch.save(embeddings, save_path)
    return embeddings

In [40]:
# Extract embeddings
train_image_embeddings = extract_image_embeddings(
    TRAIN_IMAGE_FOLDER, train_df, "train_image_embeddings.pt", vision_model, vision_processor
)
val_image_embeddings = extract_image_embeddings(
    VAL_IMAGE_FOLDER, val_df, "val_image_embeddings.pt", vision_model, vision_processor
)
test_image_embeddings = extract_image_embeddings(
    TEST_IMAGE_FOLDER, test_df, "test_image_embeddings.pt", vision_model, vision_processor
)

Embeddings already exist at train_image_embeddings.pt
Embeddings already exist at val_image_embeddings.pt
Embeddings already exist at test_image_embeddings.pt


In [41]:
def extract_text_embeddings(df, save_path, model, tokenizer):
    if os.path.exists(save_path):
        print(f"Embeddings already exist at {save_path}")
        return torch.load(save_path)

    embeddings = {}
    model.eval()
    with torch.no_grad():
        for idx, row in tqdm(df.iterrows(), desc="Extracting text embeddings", total=len(df)):
            transcription = row["transcriptions"]
            transcription = transcription if isinstance(transcription, str) else ""
            
            # Tokenize the text
            inputs = tokenizer(
                transcription, padding="max_length", truncation=True, max_length=128, return_tensors="pt"
            )
            inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU/CPU
            
            # Extract embeddings
            outputs = model(**inputs)
            embeddings[row["image_id"]] = outputs.last_hidden_state[:, 0, :].cpu()

    torch.save(embeddings, save_path)
    return embeddings

In [42]:
train_text_embeddings = extract_text_embeddings(
    train_df, "train_text_embeddings.pt", text_model, text_tokenizer
)
val_text_embeddings = extract_text_embeddings(
    val_df, "val_text_embeddings.pt", text_model, text_tokenizer
)
test_text_embeddings = extract_text_embeddings(
    test_df, "test_text_embeddings.pt", text_model, text_tokenizer
)

Extracting text embeddings: 100%|██████████| 356/356 [38:13<00:00,  6.44s/it]
